# A Simple Tutorial - How to Handle Missing Data ?

Data in real world are rarely clean and homogeneous. Typically this is because of the following reasons

1. They tend to be incomplete

2. Noisy, and inconsistent 

3. Corrupt data

4. Failure to load information

5. Incomplete extraction

So it is an important task of a Data scientist to prepossess the data by filling missing values because making the right decision on how to handle it generates robust data models. It is important to be handled as they could lead to wrong prediction or classification for any given model being used. The goal of this article is to cover the basic techniques for handling missing values in a dataset.

Real-world data often has missing values. 

Let us look at the titanic dataset example as a good starting point to under this concept in detail.

Specifically, after completing this tutorial you will know:

- How to marking invalid or corrupt values as missing in your dataset.
- How to remove rows with missing data from your dataset.
- How to impute missing values with mean values in your dataset.

Let’s get started.

First let us load the data 

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# Load Data
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
# Concatenate train & test
train_objs_num = len(train)
y = train['Survived']
dataset = pd.concat(objs=[train.drop(columns=['Survived']), test], axis=0)

Now let us find out the missing values.

In [ ]:
dataset.info()

We can easily understand from above that features like ‘Age’, ‘Fare’, ‘Cabin’ and ‘Embarked’ contains missing values. Lets analyse deeper to look at in detail

In [ ]:
total = dataset.isnull().sum().sort_values(ascending=False)
percent = (dataset.isnull().sum()/dataset.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
f, ax = plt.subplots(figsize=(15, 6))
plt.xticks(rotation='90')
sns.barplot(x=missing_data.index, y=missing_data['Percent'])
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)
missing_data.head()

Let us look now at some sample observations in the dataset to see how the data is looking like .

In [ ]:
dataset.head()

In the above sample it is observed that NaN are present.

In general missing values could be: NaN, empty string, ?,-1,-99,-999 and so on. In order to understand if -1 is a missing value or not we could draw a histogram. If this variable has a uniform distribution between 0 and 1 and it has a small peak at -1 then -1 is actually a missing value.
Missing values can be hidden from us and by hidden mean replaced by some other value beside NaN. Therefore, it is always beneficial to plot a histogram in order to identify those values.

So as an example let us plot a histogram for Age feature as it is a numerical feature.

In [ ]:
# Plot histogram using seaborn
plt.figure(figsize=(15,8))
sns.distplot(dataset.Age, bins =30)

Firstly we cannot simply ignore missing values in a dataset. We must handle them in some way for the very practical reason that most algorithms do not accept missing values.

"Common sense" is not sensible here.From my experience the 2 most commonly recommended ways of dealing with missing data actually are not accurate .

They are:

1. Dropping observations that have missing values
2. Imputing the missing values based on other observations

Dropping missing values is sub-optimal because when we drop observations, we drop information.

The fact that the value was missing may be informative in itself.We need to understand business deeper to uncover why this information is missing in real world problems.In real time problems we need to make predictions even if some of the features are missing !!!.


Imputing missing values is sub-optimal because the value was originally missing but you filled it in, which always leads to a loss in information, no matter how sophisticated our imputation method is.

"<b>Missingness</b>" is almost always informative in itself, and we should tell our algorithm if a value was missing.Even if we build a model to impute our values, we are not adding any real information. You’re just reinforcing the patterns already provided by other features.

Basically, **there are three categories of missing data**. We assume that each record or observation can be divided into an "observable component" and an "unobservable component". We also assume that the records are independent and identically distributed.

1. **MCAR (Missing Completely At Random)** where the pattern of missinginess is statistically independent of the data record. Example: you have a data set on a piece of paper and you spill coffee on the paper destroying part of the data.

2. **MAR (Missing At Random)** where the probability distribution of the pattern of missingness is functionally dependent upon the observable component in the record. MCAR is a special case of MAR. Example: you have a question on a survey asking if the survey participant is a drug addict and another question which asks if the survey participant has less than one alcoholic drink per year. Assume the answer to the alcoholic drink question is always observable, then the probability that someone fails to answer the drug addict question is most likely functionally dependent upon their answer to the alcoholic drink question.

3. **MNAR (Missing Not at Random)** which is defined as the case which is NOT MAR. In the MNAR case, you can have situations where both the drug addict and alcoholic drink questions are absent in the same record. Another example, is a case where the question is: "What is your gender?" Suppose that females are less likely to answer this question than males. This is another example of an MNAR question because the probability that the answer is observable is conditionally dependent upon the unobservable component of the data record.

Now that we know the basic terminology,I like to share some strategies and recommendations

**Strategies** :

1. We should never insert mean, mode, median, max, min or anything else for missing values. That is, avoid deterministic imputation even though it is widely used and available in most software packages. It underestimates and distorts the statistical regularities (e.g., underestimates variance is one example) present in the data sample. 

2. If the data records are MCAR Then you can delete records with missing data.

3. If the data records are MCAR, then sometimes you can stochastically impute the missing values rather than deterministically impute them. So this means that if you specify the marginal probability distribution of a missing value as Gaussian with some known mean and some known variance then you can sample from that distribution to impute values into the data set. We need to be careful and do some additional research  and analysis on data ,understand the business completely and take a judicious decision.

4. If the data is MAR then an algorithm such as Expectation Maximization can be used to handle the missing observations.

5. If the data is MNAR we can include binary indicators in the data record which explicitly identify when a variable is not observable. The challenge with this approach is that a highly nonlinear model needs to be designed to properly integrate this information in an appropriate manner. This might work in a machine learning algorithm where the binary indicators "disconnect" the influence of predictors which are not observable. Consequently, the MNAR theory (i.e., the theory of the joint distribution of the complete data record and missing data pattern) is instantiated in the learning machine's probabilistic model of its statistical environment.


#### Missing numeric data

For missing numeric data, we should flag and fill the values.

1. Flag the observation with an indicator variable of missingness.
2. Secondly fill the original missing value with 0 just to meet the technical requirement of no missing values.

By using this technique of flagging and filling, we are essentially allowing the algorithm to estimate the optimal constant for missingness, instead of just filling it in with the mean.

#### Missing categorical data

The best way to handle missing data for categorical features is to simply label them as ’Missing’!

1. We are essentially adding a new class for the feature.
2. This tells the algorithm that the value was missing.
3. This also gets around the technical requirement for no missing values.

### Missing values Treatment Techniques

#### 1.Deleting Data 
This method is the most commonly used to handle the null values. Here, we either delete a particular record if it has a null value for a particular feature ,and a particular feature if it has more than 70-75% of missing values. This method is advised only when there are enough samples in the data set. 

Note : One has to make sure that after we have deleted the data, there is no addition of bias. Removing the data will lead to loss of information which will not give the expected results while predicting the output.

Imagine we drop one whole observation just because one of the features had a missing value, even if the rest of the features are perfectly filled and informative!

In [ ]:
df = dataset
# Will drop all features with missing values 
df.dropna(inplace = True)
df.isnull().sum()

You can also select to drop the rows only if all of the values in the row are missing.

In [ ]:
df1 = dataset
# Will drop the rows only if all of the values in the row are missing
df1.dropna(how = 'all',inplace = True)

Sometimes, we may just want to drop a feature that has some missing values.

In [ ]:
df = dataset
# Will drop a feature that has some missing values.
df.dropna(axis = 1,inplace = True)

Suppose we want to keep only the rows with at least 4 non-na values

In [ ]:
df = dataset
# Keep only the rows with at least 4 non-na values
df.dropna(thresh = 4,inplace = True)

In summary it is always better to keep data than to delete them.

**The only case that it may worth deleting a variable is when its missing values are more than 60% of the observations but only if that variable is insignificant.Taking this into consideration, imputation is always a preferred choice over deleting variables.**

**Pros:**
- Complete removal of data with missing values results in robust and highly accurate model
- Deleting a particular row or a column with no specific information is better, since it does not have a high weightage

**Cons:**
- Loss of information and data
- Works poorly if the percentage of missing values is high (say 30%), compared to the whole dataset

#### 2. Back-fill or Forward-fill
Back-fill or forward-fill to propagate next or previous values respectively.

In [ ]:
df = dataset
#for back fill
df.fillna(method='bfill',inplace=True)
#for forward-fill
df.fillna(method='ffill',inplace=True)

** Key Note:** NaN value will remain even after forward filling or back filling if a next or previous value isn’t available or it is also a NaN value.

#### 3. Replace with constant value

Replace with some constant value outside fixed value range-999,-1 etc
This method is useful as it gives the possibility to group missing values as a separate category represented by a constant value. It is a preferred option when it doesn’t make sense to try and predict a missing value. The downside is that performance of linear models can suffer.Use a global constant to fill in for missing values.
For example, in the titanic dataset filling in the missing value of the Embarked feature with the most common Port of Embarkation might not really makes sense as opposed to using something like “N/A”.

In [ ]:
# Replace with a constant value
# dataframe.Column_Name.fillna(-99,inplace=True)

#### 4. Replacing With Mean/Median/Mode

This strategy can be applied on a feature which has numeric data like the age of a person or the ticket fare. We can calculate the mean, median or mode of the feature and replace it with the missing values. This is an approximation which can add variance to the data set. But the loss of the data can be negated by this method which yields better results compared to removal of rows and columns. Replacing with the above three approximations are a statistical approach of handling the missing values. This method is also called as** leaking the data while training**. Another way is to approximate it with the deviation of neighbouring values. This works better if the data is linear.  

**MEAN:** Suitable for continuous data without outliers

In [ ]:
df3 = train
df3['Age'].isnull().sum()

In [ ]:
df3['Age'].mean()

In [ ]:
df3['Age'].replace(np.NaN,df3['Age'].mean()).head(15)

As you see above in the fifth row the mean value is replaced in place of NaN.

**MEDIAN :** Suitable for continuous data with outliers

In [ ]:
df4 = train
df4['Age'].fillna(df4['Age'].median(),inplace=True)
df4.head()

**Mode:** For categorical feature we can select to fill in the missing values with the most common value(mode) as illustrated below.

In [ ]:
data_cat=train
data_cat['Embarked'].fillna(data_cat['Embarked'].mode()[0], inplace=True)
data_cat.head()

**Pros:**
* This is a better approach when the data size is small
* It can prevent data loss which results in removal of the rows and columns

**Cons:**
- Imputing the approximations add variance and bias
- Works poorly compared to other multiple-imputations method


**Note:** Mean, Median and Mode imputation diminishes any correlations involving the variable(s) that are imputed. This is because we assume that there is no relationship between the imputed variable and any other measured variables. Thus, those imputations have some attractive properties for univariate analysis but become problematic for multivariate analysis.

#### 5. Assigning An Unique Category

A categorical feature will have a definite number of possibilities, such as gender, for example. Since they have a definite number of classes, we can assign another class for the missing values. Here, the features Cabin and Embarked have missing values which can be replaced with a new category, say, U for ‘unknown’. This strategy will add more information into the dataset which will result in the change of variance. Since they are categorical, we need to find one hot encoding to convert it to a numeric form for the algorithm to understand it. 

In [ ]:
data_unique = train
data_unique['Cabin'].head(10)

In [ ]:
data_unique['Cabin'].fillna('U').head(10)

**Pros:**

* Less possibilities with one extra category, resulting in low variance after one hot encoding — since it is categorical
* Negates the loss of data by adding an unique category

**Cons:**

* Adds less variance
* Adds another feature to the model while encoding, which may result in poor performance


## Summary

The take-way from this kernel is:

* The approach to deal with missing values is heavily dependent on the nature of the dataset
* It is always useful to ask, why there are missing values?Understand the business context.
* Different ways of handling missing data have difference effects on the performance of models.

# If you like this kernel greatly appreciate an UPVOTE 